In [1]:
%pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
# YAML
import os

# Roboflow кладёт YAML рядом с изображениями/лейблами
data_yaml = r"""/home/jupyter/datasphere/project/ЛЦТ_seg-19/data.yaml"""
assert os.path.isfile(data_yaml), f"Не найден data.yaml по пути: {data_yaml}"

print("YAML:", data_yaml)

YAML: /home/jupyter/datasphere/project/ЛЦТ_seg-19/data.yaml


In [ ]:
# ==============================
# Обучение YOLOv11l (Detection) — public ckpt
# ==============================

from ultralytics import YOLO

# 1) Публичный чекпоинт YOLOv11-L
model = YOLO("yolo11l.pt")

# 2) Динамика «составных» аугментаций
def _schedule(trainer):
    e, E = trainer.epoch, trainer.epochs
    p = e / max(E, 1.0)

    if p < 0.4:
        trainer.args.mosaic = 0.30
        trainer.args.copy_paste = 0.25
        trainer.args.mixup = 0.05
    elif p < 0.8:
        trainer.args.mosaic = 0.15
        trainer.args.copy_paste = 0.15
        trainer.args.mixup = 0.02
    else:
        trainer.args.mosaic = 0.00
        trainer.args.copy_paste = 0.00
        trainer.args.mixup = 0.00

model.add_callback("on_train_epoch_start", _schedule)

results = model.train(
    data=data_yaml,
    epochs=140,
    imgsz=832,
    batch=-1,
    device=0,
    pretrained=True,
    amp=True,
    cos_lr=True,

    optimizer="AdamW",
    lr0=0.003,
    lrf=0.20,
    weight_decay=0.01, 
    nbs=64,

    warmup_epochs=3.0,
    warmup_momentum=0.80,
    warmup_bias_lr=0.10,

    label_smoothing=0.05,

    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    translate=0.08,
    scale=0.50,
    degrees=0.0,
    shear=0.0,
    perspective=0.0,
    fliplr=0.0,
    flipud=0.0,

    mosaic=0.30, copy_paste=0.25, mixup=0.05,
    close_mosaic=20,

    cache=True,
    workers=16,
    seed=42,
    max_det=500,
    plots=True,
    name="yolo11_L_7k"
)

New https://pypi.org/project/ultralytics/8.3.204 available 😃 Update with 'pip install -U ultralytics'
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in the future.
Ultralytics 8.3.203 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-80GB, 81229MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=20, cls=0.5, compile=False, conf=None, copy_paste=0.25, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/home/jupyter/datasphere/project/ЛЦТ_seg-19/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=140, erasing=0.4, exist_ok=False, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=832, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.003, lrf=0.2, mask_ratio=4, max_det=500, mixup=0.05, mode